Taller 2 FC

Estudiantes: 

Laura Sofía Calderón Sterling

Janny Valentina Corredor Mora


In [1]:
#Importamos librerias
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import linprog

Punto 1. Encuentre la asignación óptima de A y B que máximizan las ganancias de la empresa.

In [2]:
#Para ver la matriz
cantidades = pd.DataFrame({' ' : ['Utilidad', 'Materia Prima', 'Mano de Obra',"Capital"],
        'Product A' : [3,1,2,1],
        'Product B' : [5, 1, 4,2]})
cantidades

,,Product A,Product B
0,Utilidad,3,5
1,Materia Prima,1,1
2,Mano de Obra,2,4
3,Capital,1,2


In [3]:
c=np.array([3,5])
A=np.array([[1,1],[2,4],[1,2],[-1,0],[0,-1]])
b=np.array([500,1000,2000,-100,-150])
#A_ub tiene que ser menor o igual, entonces le cambio el signo 
solucion=linprog(-c,A_ub=A,b_ub=b,method="revised simplex")
pd.DataFrame(solucion.x)
#Como Ax1+bX2>=150 y >=100 entonces cambio los coeficientes en la matriz B

<ipython-input-3-b24ed0c48646>:5: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  solucion=linprog(-c,A_ub=A,b_ub=b,method="revised simplex")


,0
0,200.0
1,150.0


Esto quiere decir que se deben producir 200 unidades de A y 150 unidades de B, para obtener la utilidad máxima, que es de $1350USD

Punto 2. Encuentre la asignación factible con el retorno esperado más alto, bajo el supuesto de que solo se permite mantener posiciones largas en los fondos.

In [4]:
c1=np.array([0.1,0.15,0.16,0.08])
A1=np.array([[1,1,1,1]])
D=np.array([[0.3,0.1,0.4,0.2],
            [0.5,0.3,0.25,0.6],
           [0.2,0.6,0.35,0.2]])
d=np.array([0.3,0.35,0.15])
b1=np.array([1])
ans=linprog(-c1,A_ub=-D,b_ub=-d,A_eq=A1,b_eq=b1,method="simplex")
pd.DataFrame(ans.x)

<ipython-input-4-6ae5c1f5bb53>:8: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  ans=linprog(-c1,A_ub=-D,b_ub=-d,A_eq=A1,b_eq=b1,method="simplex")


,0
0,0.000000
1,0.157895
2,0.578947
3,0.263158


In [5]:
c1=np.array([0.1,0.15,0.16,0.08])
A1=np.array([1,1,1,1])
D=np.array([[0.3,0.1,0.4,0.2],
            [0.5,0.3,0.25,0.6],
           [0.2,0.6,0.35,0.2]])
d=np.array([0.3,0.35,0.15])
b1=np.array([100])

In [6]:
#Ojo porque acá todas son de desigualdad, menos el 100
n=4
x = cp.Variable(n)
objective=cp.Minimize(-c1.T@x)
constraints=[cp.sum(x)==1, D@x>=d,x>=0] #A1@x==b1
prob = cp.Problem(objective,constraints)
result=prob.solve()
result

-0.1373684208709285

In [7]:
x_sol=pd.DataFrame(x.value)*100
print(round(x_sol.T,4))

     0        1        2        3
0  0.0  15.7895  57.8947  26.3158


Para maximizar el retorno es necesario invertir 15.789  millones en el fondo 2, 57.894 millones en el fondo 3 y 26.315 millones en el fondo 4. 

Al invertir esa suma el retorno de la inversión será 13.737 millones

Punto 3. Determine si usar la línea de crédito en algunos meses le representa un alto costo, de ser así determine cuando usarla, cuanto cuesta y que pagos de intereses tendrá que hacer entre enero y junio.

In [8]:
#Ojo usa SCIPY Y CVXPY
c3=np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1])
#Son 14 variables
A3=np.array([[1,0,0,0,0, 1,0,0, -1,0,0,0,0,0],
             [-1.01,1,0,0,0, 0,1,0, 1.003,-1,0,0,0,0],
             [0,-1.01,1,0,0, 0,0,1, 0,1.003,-1,0,0,0],
             [0,0,-1.01,1,0, -1.02,0,0, 0,0,1.003,-1,0,0],
             [0,0,0,-1.01,1, 0,-1.02,0, 0,0,0,1.003,-1,0],
             [0,0,0,0,-1.01, 0,0,-1.02, 0,0,0,0,1.003,-1]])
b3=np.array([150,100,-200,200,-50,-300])
#Como necesito financiar va positivo, porque las quiero minimizar
#EQ SIEMPRE ES DE IGUALDAD Y UB ES DE DESIGUALDAD
#Si es menor o igual lo dejo positivo, porque es un límite superior 
res3=linprog(-c3,A_eq=A3, b_eq=b3,method="revised simplex")
res3

<ipython-input-8-062b3db406f5>:14: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res3=linprog(-c3,A_eq=A3, b_eq=b3,method="revised simplex")


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -92.49694915254233
       x: [ 0.000e+00  5.098e+01 ...  0.000e+00  9.250e+01]
     nit: 6

El mismo punto 3 pero con GLPK

In [9]:
n=14
x = cp.Variable(n)
objective=cp.Minimize(-c3.T@x)
#Como todo es igualdad entonces va con doble igual
constraints=[A3@x==b3, x[0:4] <=100, x >=0]
prob = cp.Problem(objective,constraints)
result=prob.solve(solver=cp.GLPK_MI)
#NECESITO EL OPTIMIZADOR DE GLPK ENTONCES LE PONGO EL SOLVER
x_sol=pd.DataFrame(x.value)
round(x_sol.T,2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.0,50.98,-0.0,-0.0,-0.0,150.0,49.02,203.43,-0.0,-0.0,351.94,-0.0,-0.0,92.5


Para maximizar el saldo de los fondos a junio (el sexto mes) es necesario pedir un crédito de 50.98 millones y emitir un bono de 49.02 millones en el mes 2; emitir un bono por 150 millones en el primer mes y uno de 203.43 millones en el mes tres. 

Estas formas de financiación implican que en el tercer mes habrá un saldo en caja de 351.94 millones.

En este caso el saldo final sería de 92.4969 millones

Punto 4. Maximice la rentabilidad de los saldos en su cuenta teniendo en cuentas las restricciones de sus flujos de efectivo.

In [10]:
#La tasa de retorno semanal periódica asumiendo 7 días
rn=(1+0.0001)**7
rn

1.0007002100350033

In [11]:
#El vector B es la diferencia entre entradas y salidas
b4=np.array([500-350,300-400,200-250,400-200])
#Como trabaje con diferencias el ento es 0.01%
rn=(1+0.0001)**7
A4=np.array([[1,0,0,0,-1,0,0,0],
             [0,1,0,0,rn,-1,0,0],
             [0,0,1,0,0,rn,-1,0],
             [0,0,0,1,0,0,rn,-1]])
c4=np.array([0,0,0,0,0,0,0,1])
#Coloco 4 x y 4 z porque uno es el valor faltante y el otro es el sobrante
res4=linprog(-c4,A_ub=A4,b_ub=-b4,method="revised simplex")
#Le pongo signo negativo a b porque es un saldo a favor que quiero maximizar y que está rentando
r=pd.DataFrame(res4.x)
round(r,2)
#Los x son los saldos que quedan que no están dando ninguna rentabilidad
#Los primeros x es una cuenta de verificación 

<ipython-input-11-c0ff07061f6a>:11: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res4=linprog(-c4,A_ub=A4,b_ub=-b4,method="revised simplex")


,0
0,0.00
1,0.00
2,0.00
3,0.00
4,150.00
5,50.11
6,0.14
7,200.14


In [12]:
np.transpose(r)

,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,150.0,50.105032,0.140116,200.140214


En este caso se puede concluir que no hay ningún monto que no este rentando. Además, para la primera semana hay una inversión de 150millones, que para final de semana renta 0.11millones. 

Al iniciar la segunda semana se hace un pago de 100 millones, por tal motivo es necesario retirarlos, al culimnar esta semana, la rentabilidad obtenida fue de 0.14 millones. 

Para la tercera semana se retiran 50 millones, con el fin de pagar las obligaciones de ese mes y, finalmente, en la última semana hay un excedente de 200 millones, los cuales son depositados de nuevo en el banco, con el fin de que sigan rentando. Todo esto, permite tener una ganancia de 0.14 millones al final de la cuarta semana


Punto 5. Maximice las ganancias de la empresa

In [13]:
c5=np.array([0,0,0,5,8,10])
#Coloco el 0 porque no me dice nada el enunciado
A5=np.array([[0,0,0,1,1,1],[1,1,1,0,0,0]])
D5=np.array([[0,0,0,0,1,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,0,1]])
d5=np.array([2000,300,250,2000,2000])
b5=np.array([10000,1000])
res5=linprog(-c5,A_ub=-D5,b_ub=-d5,A_eq=A5,b_eq=b5,method="simplex")
res5

<ipython-input-13-bf1876eefac6>:7: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res5=linprog(-c5,A_ub=-D5,b_ub=-d5,A_eq=A5,b_eq=b5,method="simplex")


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -86000.0
       x: [ 4.500e+02  3.000e+02  2.500e+02  2.000e+03  2.000e+03
            6.000e+03]
     nit: 3

In [14]:
r=pd.DataFrame(res5.x)
r

,0
0,450.0
1,300.0
2,250.0
3,2000.0
4,2000.0
5,6000.0


Para maximizar la utilidad obtenida por medio de la asignación de presupuesto y de horas, se debe destinar 6000 millones y 250 horas al departamento de producción, 2000 millones y 300 horas a marketing y, por último, 2000 millones y 450 horas al departamento de ventas. 

Con estas asignaciones la ganancia resultante es de 86000 millones. 

Punto 6. ¿Cuál es la estrategia óptima para reducir los costos de funcionamiento y transporte de la empresa?

In [15]:
Costos = pd.DataFrame({'Planta' : ['1', '2', '3'],
        'Costo 1' : [4,6,9],
        'Costo 2' : [5,4,7], 
        'Costo 3' : [6,3,4],
        'Costo 4' : [8,5,3], 
        'Costo 5' : [10,8,4],
        'Fj' : [1000,1000,1000],
         'Mj' : [500,500,500]})
Costos


,Planta,Costo 1,Costo 2,Costo 3,Costo 4,Costo 5,Fj,Mj
0,1,4,5,6,8,10,1000,500
1,2,6,4,3,5,8,1000,500
2,3,9,7,4,3,4,1000,500


In [16]:
demanda=pd.DataFrame({"Puntos": ['1', '2', '3',"4","5"],
                      "Demanda":[80,270,250,160,180]})
demanda

,Puntos,Demanda
0,1,80
1,2,270
2,3,250
3,4,160
4,5,180


In [17]:
c6 = np.array([[4, 6, 9], [5, 4, 7], [6, 3, 4], [8, 5, 3], [10, 8, 4]]) #Coeficientes función costos
D6 = np.array([80, 270, 250, 160, 180]) #Coeficientes demanda
F6 = np.array([1000, 1000, 1000]) #Coeficientes Fj
M6 = np.array([500, 500, 500]) #Coeficientes Mj

In [18]:
x = cp.Variable((5, 3), nonneg=True)
y = cp.Variable(3, boolean=True)

objective = cp.Minimize(cp.sum(cp.multiply(F6, y)) + cp.sum(cp.multiply(c6, x)))

In [19]:
constraints = []

for i in range(5):
  constraints.append(cp.sum(x[i, :]) >= D6[i])

for j in range(3):
  constraints.append(cp.sum(x[:, j]) <= M6[j] * y[j])



In [20]:
prob = cp.Problem(objective,constraints)
result=prob.solve()
result

5610.0

In [21]:
rta=pd.DataFrame(x.value)
round(rta,2)

,0,1,2
0,0.0,80.0,-0.0
1,-0.0,270.0,-0.0
2,-0.0,150.0,100.0
3,-0.0,-0.0,160.0
4,-0.0,-0.0,180.0


In [22]:
np.transpose(rta)

,0,1,2,3,4
0,0.0,-0.0,-0.0,-0.0,-0.0
1,80.0,270.0,150.0,-0.0,-0.0
2,-0.0,-0.0,100.0,160.0,180.0


Para optimizar los costos de transporte y funcionamiento la empresa deberia cerrar la planta 1, ya que con la planta 2 y 3 puede distribuir la cantidad demandada y se incurririan en menores costos. 

En este sentido, la asignación de producción sería así: La planta 1 estaría cerrada, entonces la producción asignada sería 0. Por su parte, la planta 2, distribuiría toda la demanda del punto de venta 1 y 2, para el punto de venta 3 distribuiría el 60%, es decir, 80, 270 y 150, respectivamente. 

Del mismo modo, la planta 3 atendería el 40% de la demanda del punto de venta 3 y la demanda total de los puntos de venta 4 y 5, entonces se encargaría de 100,160 y 180, respectivamente.